# Homework 6 - Generative Adversarial Network
This is the sample code for hw6 of 2022 Machine Learning course in National Taiwan University. 

In this sample code, there are 5 sections:
1. Environment setting
2. Dataset preparation
3. Model setting
4. Train
5. Inference

Your goal is to do anime face generation, if you have any question, please discuss at NTU COOL 

# Environment setting
In this section, we will prepare for the dataset and set some environment variable

## Download Dataset

In [ ]:
# ! rm -rf ./MLHW_6

In [ ]:
# get dataset from huggingface hub
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/datasets/LeoFeng/MLHW_6
!unzip ./MLHW_6/faces.zip -d .


## Other setting

In [ ]:
# import module
import os
import glob
import random
from datetime import datetime

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch import optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

import matplotlib.pyplot as plt
import numpy as np
import logging
from tqdm import tqdm


# seed setting
def same_seeds(seed):
    # Python built-in random module
    random.seed(seed)
    # Numpy
    np.random.seed(seed)
    # Torch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

same_seeds(2022)
workspace_dir = '.'

In [ ]:
!pip install stylegan2_pytorch

In [ ]:
!stylegan2_pytorch --new --name style --data ./faces --image-size 64 --num-train-steps 25000 --network-capacity 24

In [ ]:
! mkdir ./images

In [ ]:
import torch
from torchvision.utils import save_image
from stylegan2_pytorch import ModelLoader
from tqdm import tqdm

loader = ModelLoader(
    base_dir = '.',   # path to where you invoked the command line tool
    name = 'style'                   # the project name, defaults to 'default'
)

for i in tqdm(range(1,1001)):
    noise   = torch.randn(1, 512).cuda() # noise
    styles  = loader.noise_to_styles(noise, trunc_psi = 0.7)  # pass through mapping network
    images  = loader.styles_to_images(styles) # call the generator on intermediate style vectors

    save_image(images, f'./images/{i}.jpg') #

In [ ]:
! cd ./images  && tar -zcvf ../images.tgz *.jpg